In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [74]:
search = input("What position are you looking for?: ") #if input is None, url will provide all positions

What position are you looking for?: 


In [75]:
url = f"https://luminorbank.teamtailor.com/jobs?query={search}"
page = requests.get(url).text

doc = BeautifulSoup(page, "html.parser")

In [76]:
jobresults = doc.find_all("li")[4:-6]

In [77]:
positions = {}
i=0

for job in jobresults:
  i+=1
  positiontitle = job.span.get("title")
  link = job.a.get("href")
  division = job.find(class_="mt-1 text-md").find_all("span")[0].text
  location = job.find(class_="mt-1 text-md").find_all("span")[2].text

  try:
        environment = job.find(class_="mt-1 text-md").find_all("span")[4].text.strip()
  except:
        environment = "On-site"

  positions["position{0}".format(i)] = {'title': positiontitle, 'link': link, 'location': location, 'environment': environment, 'division': division}

In [78]:
positions

{'position1': {'division': 'Legal',
  'environment': 'Hybrid Remote',
  'link': 'https://luminorbank.teamtailor.com/jobs/1833201-legal-counsel',
  'location': 'Tallinn, Estonia ',
  'title': 'Legal Counsel'},
 'position10': {'division': 'Retail Banking',
  'environment': 'Hybrid Remote',
  'link': 'https://luminorbank.teamtailor.com/jobs/1826535-transactions-monitoring-specialist',
  'location': 'Vilnius, Lithuania ',
  'title': 'Transactions Monitoring Specialist'},
 'position11': {'division': 'Compliance',
  'environment': 'Hybrid Remote',
  'link': 'https://luminorbank.teamtailor.com/jobs/1768217-project-manager',
  'location': 'Tallinn, Estonia ',
  'title': 'Project Manager'},
 'position12': {'division': 'Compliance',
  'environment': 'Hybrid Remote',
  'link': 'https://luminorbank.teamtailor.com/jobs/1768180-project-manager',
  'location': 'Vilnius, Lithuania ',
  'title': 'Project Manager'},
 'position13': {'division': 'Compliance',
  'environment': 'Hybrid Remote',
  'link': 'h

In [79]:
jobs_df = pd.DataFrame.from_dict(positions, orient='index', columns=['title', 'location', 'environment', 'division', 'link']) #creating dataframe

In [80]:
jobs_df

,title,location,environment,division,link
position1,Legal Counsel,"Tallinn, Estonia",Hybrid Remote,Legal,https://luminorbank.teamtailor.com/jobs/183320...
position2,Senior Business Analyst,"Vilnius, Lithuania",Hybrid Remote,Corporate Banking,https://luminorbank.teamtailor.com/jobs/183318...
position3,Senior Business Analyst,"Riga, Latvia",Hybrid Remote,Corporate Banking,https://luminorbank.teamtailor.com/jobs/183314...
position4,Senior Business Analyst,"Tallinn, Estonia",Hybrid Remote,Corporate Banking,https://luminorbank.teamtailor.com/jobs/183310...
position5,Cash Management Specialist,Multiple locations,Hybrid Remote,Corporate Banking,https://luminorbank.teamtailor.com/jobs/183292...
...,...,...,...,...,...
position69,Marketing Project Manager,"Vilnius, Lithuania",On-site,Retail Banking,https://luminorbank.teamtailor.com/jobs/175690...
position70,Klientu risinājumu konsultants,"Riga, Latvia",On-site,Retail Banking,https://luminorbank.teamtailor.com/jobs/142400...
position71,AML Specialist (EDD group),"Vilnius, Lithuania , Kaunas, Lithuania",Hybrid Remote,Retail Banking,https://luminorbank.teamtailor.com/jobs/155756...
position72,Konsultantas (-ė) Kliento aptarnavimo skyriuje...,"Vilnius, Lithuania",Hybrid Remote,Retail Banking,https://luminorbank.teamtailor.com/jobs/162850...


In [81]:
page_results =[]
for link in jobs_df['link']:
  posURL = f"{link}"
  pos_page = requests.get(posURL).text

  pos_doc = BeautifulSoup(pos_page, "html.parser")
  page_results.append(pos_doc.find_all(class_="mx-auto max-w-[750px] prose font-company-body overflow-hidden"))

In [82]:
similarity = []

my_knowledge = ["Python", "SQL", "can-do attidute", "Machine Learning", "Data Analysis", "Degree", "Master's", "Bachelor", "English", "Ability to learn", "English"] #person-specific knowledge
  

for page in range(len(jobs_df)):
  contents = page_results[page][0].find_all("li")
  count = 0
  for content in contents:
    try:
      for knowledge in my_knowledge:
        if knowledge.lower() in content.text.lower():
          count += 1
    except:
        continue
  similarity.append(count)

In [83]:
similarity_perc = []
for item in similarity:
  similarity_perc.append(item/(len(my_knowledge)))

In [84]:
jobs_df["Similarity"] = similarity_perc #adding a column - similarity

In [90]:
jobs_df.nlargest(n=1, columns=['Similarity'])

,title,location,environment,division,link,Similarity
position14,Internship in Macro Modelling area,"Tallinn, Estonia",Hybrid Remote,Risk,https://luminorbank.teamtailor.com/jobs/182414...,0.545455


In [91]:
jobs_df.to_csv("luminor-jobs.csv", index=True)